In [1]:
#import the packages that we might use
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor 
from sklearn.linear_model import ElasticNet,SGDRegressor,BayesianRidge,LogisticRegression,LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn import utils

In [2]:
#Show the packages' names and the versions
print('python version:'+sys.version)
print('packages:\n'+'\n'.join(f'{m.__name__}={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

python version:3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]
packages:
pandas=1.3.5
numpy=1.22.3


<font face="Times New Roman" size=5 color=#000000 > 
Load the Data  <br/>
<font face="Times New Roman" size=3 color=#000000 >  
We use 2018-2020 Marketscan data to analyse and we extract the relative rows'data in 3 year's 'mdcro.csv'.

In [3]:
#Read 'mdcro201.csv' data from marketscan
df = pd.read_csv('mdcro201.csv',nrows = 1000000,low_memory=False)

In [4]:
#Show the shape of the data
df.shape

(1000000, 58)

In [5]:
#Show the first 10 rows'data in the dataset
df.head(n = 10)

,SEQNUM,VERSION,DX1,DX2,PROC1,PROCTYP,EFAMID,ENROLID,REVCODE,SVCDATE,...,ENRFLAG,PHYFLAG,RX,SEX,HLTHPLAN,INDSTRY,MSCLMID,NPI,UNITS,MEDADV
0,19366318,10,R05,NaN,71046,1.0,19325815,NaN,NaN,01/01/2020,...,1,1,1,2.0,0,7.0,1585629,NaN,1.0,0
1,22391333,10,Z1211,NaN,82274,1.0,20350523,NaN,NaN,01/01/2020,...,1,0,1,NaN,0,7.0,1103743,NaN,1.0,0
2,18785199,10,M25511,NaN,73030,1.0,36816101,NaN,NaN,01/02/2020,...,1,1,1,NaN,0,2.0,11676728,yoaeNToLfc,1.0,0
3,18785200,10,M25511,NaN,73030,1.0,36816101,NaN,NaN,01/02/2020,...,1,1,1,NaN,0,2.0,11676726,ydFFfhh4ot,1.0,0
4,18785201,10,M25511,M25551,99213,1.0,36816101,NaN,NaN,01/02/2020,...,1,1,1,NaN,0,2.0,11676727,yoaeNToLfc,1.0,0
5,19366319,10,E7801,I10,99214,1.0,18203047,NaN,NaN,01/02/2020,...,1,1,1,1.0,0,7.0,1269474,yvo2Al32sc,1.0,0
6,19366320,10,G4733,NaN,E0601,7.0,18213469,NaN,NaN,01/02/2020,...,1,1,1,2.0,0,7.0,471895,NaN,1.0,0
7,19366321,10,L309,NaN,88305,1.0,18217523,NaN,NaN,01/02/2020,...,1,1,1,1.0,0,7.0,1398078,NaN,1.0,0
8,19366322,10,L309,NaN,88341,1.0,18217523,NaN,NaN,01/02/2020,...,1,1,1,1.0,0,7.0,1398078,NaN,7.0,0
9,19366323,10,L309,NaN,88342,1.0,18217523,NaN,NaN,01/02/2020,...,1,1,1,1.0,0,7.0,1398078,NaN,1.0,0


In [6]:
#Load the 'mdcro192.csv' and 'mdcro181.csv' data from marketscan 
df2 = pd.read_csv('mdcro192.csv',nrows = 1000000,low_memory=False)
df3 = pd.read_csv('mdcro181.csv',nrows = 500000,low_memory=False)

<font face="Times New Roman" size=5 color=#000000 >
    Data Preprocessing and data preprocessing <br/>
<font face="Times New Roman" size=3 color=#000000 >
   We need to do data cleaning and construct them as training sets and testing sets.

In [7]:
#Choose the data we would like to search if these are relavent to our claim
df = df[['AGE','SEX','REVCODE','PROCTYP','PAY','NETPAY','COB','COINS','COPAY','DEDUCT','INDSTRY','UNITS',
'REGION','STDPLAC','EGEOLOC','EESTATU','HLTHPLAN','QTY','RX','YEAR',
'PROC1','SVCDATE','TSVCDAT','DX1','DX2','DX3','DX4']]
df2 = df2[['AGE','SEX','REVCODE','PROCTYP','PAY','NETPAY','COB','COINS','COPAY','DEDUCT','INDSTRY','UNITS',
'REGION','STDPLAC','EGEOLOC','EESTATU','HLTHPLAN','QTY','RX','YEAR',
'PROC1','SVCDATE','TSVCDAT','DX1','DX2','DX3','DX4']]
df3 = df3[['AGE','SEX','REVCODE','PROCTYP','PAY','NETPAY','COB','COINS','COPAY','DEDUCT','INDSTRY','UNITS',
'REGION','STDPLAC','EGEOLOC','EESTATU','HLTHPLAN','QTY','RX','YEAR',
'PROC1','SVCDATE','TSVCDAT','DX1','DX2','DX3','DX4']]
df_list = [df,df2,df3]
df = pd.concat(df_list)
df.shape

(2500000, 27)

In [8]:
#For the following research, we need to find the relationship of sex and proc and also look at the code in 'PROC1'
pd.crosstab(df.PROC1, df.SEX)

SEX,1.0,2.0
PROC1,,
0001A,2,1
0001F,80,80
0005U,2,0
00100,4,5
00103,46,102
...,...,...
V5258,2,1
V5260,21,37
V5261,241,305


In [9]:
#Delete all the grid which is NAN
df = df[df['SEX'].notna()]
df = df[df['INDSTRY'].notna()]
df = df[df['UNITS'].notna()]
df = df[df['PROC1'].notna()]
df = df[df['SVCDATE'].notna()]
df = df[df['PROCTYP'].notna()]
df = df[df['REVCODE'].notna()]
df = df[df['TSVCDAT'].notna()]
df = df[df['QTY'].notna()]
df = df[df['RX'].notna()]
df = df[df['YEAR'].notna()]
df = df[df['EGEOLOC'].notna()]
df = df[df['EESTATU'].notna()]
df = df[df['HLTHPLAN'].notna()]
df = df[df['AGE'].notna()]
df = df[df['PAY'].notna()]
df = df[df['NETPAY'].notna()]
df = df.fillna(0)

In [10]:
#Caculate the claim in each row and look at the head 5 rows.
#df['CLAIM'] = df['NETPAY']
df['CLAIM'] = df['PAY'] - df['NETPAY']
df.shape

(3828, 28)

In [11]:
#Caculate the number of time when the patient is in the outpatient hospital
df['TIME'] = pd.to_datetime(df['TSVCDAT']) - pd.to_datetime(df['SVCDATE'])
df['TIME'] = df['TIME'].astype('timedelta64[D]').astype(float)
df['TIME'] = df['TIME'] + 1.0
df.head()

,AGE,SEX,REVCODE,PROCTYP,PAY,NETPAY,COB,COINS,COPAY,DEDUCT,...,YEAR,PROC1,SVCDATE,TSVCDAT,DX1,DX2,DX3,DX4,CLAIM,TIME
13,73.0,1.0,361.0,1.0,392.99,392.99,0.0,0.0,0.0,0.0,...,2020,11104,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0
14,73.0,1.0,300.0,1.0,754.72,754.72,0.0,0.0,0.0,0.0,...,2020,81340,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0
15,73.0,1.0,310.0,1.0,727.92,727.92,0.0,0.0,0.0,0.0,...,2020,81342,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0
16,73.0,1.0,310.0,1.0,174.27,174.27,0.0,0.0,0.0,0.0,...,2020,88305,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0
17,73.0,1.0,310.0,1.0,1610.00,1610.00,0.0,0.0,0.0,0.0,...,2020,88341,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0


In [12]:
#Take a look at the vital number of the features
df.describe().to_csv('describe.csv')

In [13]:
#In revenue code feature, we use the data binning in groups of 10
df['REVCODE'] = pd.cut(df['REVCODE'],78)
df['EGEOLOC'] = pd.cut(df['EGEOLOC'],7)

In [14]:
#In procedure code type feature, we use the data binning in groups of 2 and devide them into CPT-4 code and HCPC code.
#df['PROCTYP'] = pd.cut(df['PROCTYP'],2)
df = df.reset_index(drop= True)
df.head(n = 10)

,AGE,SEX,REVCODE,PROCTYP,PAY,NETPAY,COB,COINS,COPAY,DEDUCT,...,YEAR,PROC1,SVCDATE,TSVCDAT,DX1,DX2,DX3,DX4,CLAIM,TIME
0,73.0,1.0,"(353.372, 362.769]",1.0,392.99,392.99,0.0,0.0,0.0,0.0,...,2020,11104,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0
1,73.0,1.0,"(296.987, 306.385]",1.0,754.72,754.72,0.0,0.0,0.0,0.0,...,2020,81340,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0
2,73.0,1.0,"(306.385, 315.782]",1.0,727.92,727.92,0.0,0.0,0.0,0.0,...,2020,81342,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0
3,73.0,1.0,"(306.385, 315.782]",1.0,174.27,174.27,0.0,0.0,0.0,0.0,...,2020,88305,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0
4,73.0,1.0,"(306.385, 315.782]",1.0,1610.00,1610.00,0.0,0.0,0.0,0.0,...,2020,88341,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0
5,73.0,1.0,"(306.385, 315.782]",1.0,236.00,236.00,0.0,0.0,0.0,0.0,...,2020,88342,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0
6,65.0,2.0,"(503.731, 513.128]",7.0,46.66,46.66,0.0,0.0,0.0,0.0,...,2020,G0463,01/04/2020,01/04/2020,M79604,0,0,0,0.0,1.0
7,77.0,2.0,"(296.987, 306.385]",1.0,70.55,70.55,0.0,0.0,0.0,0.0,...,2020,80053,01/06/2020,01/06/2020,C23,C221,0,0,0.0,1.0
8,77.0,2.0,"(296.987, 306.385]",1.0,68.51,68.51,0.0,0.0,0.0,0.0,...,2020,82378,01/06/2020,01/06/2020,C23,C221,0,0,0.0,1.0
9,77.0,2.0,"(296.987, 306.385]",1.0,28.05,28.05,0.0,0.0,0.0,0.0,...,2020,85025,01/06/2020,01/06/2020,C23,C221,0,0,0.0,1.0


In [15]:
#Set the Procedure name code and we need to make the relationship with PROC1
df['PROC1_CATEGORIES'] = '0'
df.head(n = 14)

,AGE,SEX,REVCODE,PROCTYP,PAY,NETPAY,COB,COINS,COPAY,DEDUCT,...,PROC1,SVCDATE,TSVCDAT,DX1,DX2,DX3,DX4,CLAIM,TIME,PROC1_CATEGORIES
0,73.0,1.0,"(353.372, 362.769]",1.0,392.99,392.99,0.00,0.0,0.0,0.0,...,11104,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,0
1,73.0,1.0,"(296.987, 306.385]",1.0,754.72,754.72,0.00,0.0,0.0,0.0,...,81340,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,0
2,73.0,1.0,"(306.385, 315.782]",1.0,727.92,727.92,0.00,0.0,0.0,0.0,...,81342,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,0
3,73.0,1.0,"(306.385, 315.782]",1.0,174.27,174.27,0.00,0.0,0.0,0.0,...,88305,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,0
4,73.0,1.0,"(306.385, 315.782]",1.0,1610.00,1610.00,0.00,0.0,0.0,0.0,...,88341,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,0
5,73.0,1.0,"(306.385, 315.782]",1.0,236.00,236.00,0.00,0.0,0.0,0.0,...,88342,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,0
6,65.0,2.0,"(503.731, 513.128]",7.0,46.66,46.66,0.00,0.0,0.0,0.0,...,G0463,01/04/2020,01/04/2020,M79604,0,0,0,0.00,1.0,0
7,77.0,2.0,"(296.987, 306.385]",1.0,70.55,70.55,0.00,0.0,0.0,0.0,...,80053,01/06/2020,01/06/2020,C23,C221,0,0,0.00,1.0,0
8,77.0,2.0,"(296.987, 306.385]",1.0,68.51,68.51,0.00,0.0,0.0,0.0,...,82378,01/06/2020,01/06/2020,C23,C221,0,0,0.00,1.0,0
9,77.0,2.0,"(296.987, 306.385]",1.0,28.05,28.05,0.00,0.0,0.0,0.0,...,85025,01/06/2020,01/06/2020,C23,C221,0,0,0.00,1.0,0


In [16]:
#We can transform it to category type directly.
df['PROC1'] = df['PROC1'].astype('string')

In [17]:
# Fill the procedure category column according the CPT and HCPC materials and tuterials

for i in range(len(df['PROC1'])):
    if df.iloc[i,3] == 1.0:
        if df.iloc[i,20] >= '00100' and df.iloc[i,20]  <= '01999':
            df.PROC1_CATEGORIES[i] = 'Anesthesia'
        elif df.iloc[i,20] >= '10004' and df.iloc[i,20] <= '69990':
            df.PROC1_CATEGORIES[i] = 'Surgery'
        elif df.iloc[i,20] >= '70010' and df.iloc[i,20] <= '79999':
            df.PROC1_CATEGORIES[i] = 'Radiology Procedures'
        elif df.iloc[i,20] >= '80000' and df.iloc[i,20] <= '89398':
            df.PROC1_CATEGORIES[i] = 'Pathology and Laboratory Procedures'
        elif df.iloc[i,20] >= '90281' and df.iloc[i,20] <= '99199':
            df.PROC1_CATEGORIES[i] = 'Medicine Services and Procedures'
        elif df.iloc[i,20] >= '99200' and df.iloc[i,20] <= '99607':
            df.PROC1_CATEGORIES[i] = 'Evaluation and Management Services'
        else:
            df.PROC1_CATEGORIES[i] = 'Other CPT Code Categories'
    else:
        df.PROC1_CATEGORIES[i] = 'HCPC Code'


df.head(n = 14)

C:\Users\Lize Zhao!mlxg\AppData\Local\Temp\ipykernel_1196\132705847.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.PROC1_CATEGORIES[i] = 'Surgery'
C:\Users\Lize Zhao!mlxg\AppData\Local\Temp\ipykernel_1196\132705847.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.PROC1_CATEGORIES[i] = 'Pathology and Laboratory Procedures'
C:\Users\Lize Zhao!mlxg\AppData\Local\Temp\ipykernel_1196\132705847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,AGE,SEX,REVCODE,PROCTYP,PAY,NETPAY,COB,COINS,COPAY,DEDUCT,...,PROC1,SVCDATE,TSVCDAT,DX1,DX2,DX3,DX4,CLAIM,TIME,PROC1_CATEGORIES
0,73.0,1.0,"(353.372, 362.769]",1.0,392.99,392.99,0.00,0.0,0.0,0.0,...,11104,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,Surgery
1,73.0,1.0,"(296.987, 306.385]",1.0,754.72,754.72,0.00,0.0,0.0,0.0,...,81340,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,Pathology and Laboratory Procedures
2,73.0,1.0,"(306.385, 315.782]",1.0,727.92,727.92,0.00,0.0,0.0,0.0,...,81342,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,Pathology and Laboratory Procedures
3,73.0,1.0,"(306.385, 315.782]",1.0,174.27,174.27,0.00,0.0,0.0,0.0,...,88305,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,Pathology and Laboratory Procedures
4,73.0,1.0,"(306.385, 315.782]",1.0,1610.00,1610.00,0.00,0.0,0.0,0.0,...,88341,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,Pathology and Laboratory Procedures
5,73.0,1.0,"(306.385, 315.782]",1.0,236.00,236.00,0.00,0.0,0.0,0.0,...,88342,01/02/2020,01/02/2020,R21,L309,0,0,0.00,1.0,Pathology and Laboratory Procedures
6,65.0,2.0,"(503.731, 513.128]",7.0,46.66,46.66,0.00,0.0,0.0,0.0,...,G0463,01/04/2020,01/04/2020,M79604,0,0,0,0.00,1.0,HCPC Code
7,77.0,2.0,"(296.987, 306.385]",1.0,70.55,70.55,0.00,0.0,0.0,0.0,...,80053,01/06/2020,01/06/2020,C23,C221,0,0,0.00,1.0,Pathology and Laboratory Procedures
8,77.0,2.0,"(296.987, 306.385]",1.0,68.51,68.51,0.00,0.0,0.0,0.0,...,82378,01/06/2020,01/06/2020,C23,C221,0,0,0.00,1.0,Pathology and Laboratory Procedures
9,77.0,2.0,"(296.987, 306.385]",1.0,28.05,28.05,0.00,0.0,0.0,0.0,...,85025,01/06/2020,01/06/2020,C23,C221,0,0,0.00,1.0,Pathology and Laboratory Procedures


In [18]:
#Transform the Diagnosis code into the string.
df['DX1'] = df['DX1'].astype('string')
df['DX2'] = df['DX2'].astype('string')
df['DX3'] = df['DX3'].astype('string')
df['DX4'] = df['DX4'].astype('string')

In [19]:
#Make a new column to measure the diagnosis code
df['DIAGNOSIS_OVERLAY'] = 0

In [20]:
#Construct the column to measure the diagnosis code
for i in range(len(df['DX1'])):
    if df.iloc[i,23] != '0':
        df.DIAGNOSIS_OVERLAY[i] += 1
    if df.iloc[i,24] != '0':
        df.DIAGNOSIS_OVERLAY[i] += 1
    if df.iloc[i,25] != '0':
        df.DIAGNOSIS_OVERLAY[i] += 1
    if df.iloc[i,26] != '0':
        df.DIAGNOSIS_OVERLAY[i] += 1
df.head()

C:\Users\Lize Zhao!mlxg\AppData\Local\Temp\ipykernel_1196\101198746.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.DIAGNOSIS_OVERLAY[i] += 1
C:\Users\Lize Zhao!mlxg\AppData\Local\Temp\ipykernel_1196\101198746.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.DIAGNOSIS_OVERLAY[i] += 1
C:\Users\Lize Zhao!mlxg\AppData\Local\Temp\ipykernel_1196\101198746.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.DIAGNOSIS_OVERLAY[

,AGE,SEX,REVCODE,PROCTYP,PAY,NETPAY,COB,COINS,COPAY,DEDUCT,...,SVCDATE,TSVCDAT,DX1,DX2,DX3,DX4,CLAIM,TIME,PROC1_CATEGORIES,DIAGNOSIS_OVERLAY
0,73.0,1.0,"(353.372, 362.769]",1.0,392.99,392.99,0.0,0.0,0.0,0.0,...,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0,Surgery,2
1,73.0,1.0,"(296.987, 306.385]",1.0,754.72,754.72,0.0,0.0,0.0,0.0,...,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0,Pathology and Laboratory Procedures,2
2,73.0,1.0,"(306.385, 315.782]",1.0,727.92,727.92,0.0,0.0,0.0,0.0,...,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0,Pathology and Laboratory Procedures,2
3,73.0,1.0,"(306.385, 315.782]",1.0,174.27,174.27,0.0,0.0,0.0,0.0,...,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0,Pathology and Laboratory Procedures,2
4,73.0,1.0,"(306.385, 315.782]",1.0,1610.00,1610.00,0.0,0.0,0.0,0.0,...,01/02/2020,01/02/2020,R21,L309,0,0,0.0,1.0,Pathology and Laboratory Procedures,2


In [21]:
#Transform the type of feature 'PROC1_CATEGORIES' into string for further study
df['PROC1_CATEGORIES'] = df['PROC1_CATEGORIES'].astype('string')
df['INDSTRY'] = df['INDSTRY'].astype('string')
df.dtypes

AGE                   float64
SEX                   float64
REVCODE              category
PROCTYP               float64
PAY                   float64
NETPAY                float64
COB                   float64
COINS                 float64
COPAY                 float64
DEDUCT                float64
INDSTRY                string
UNITS                 float64
REGION                  int64
STDPLAC               float64
EGEOLOC              category
EESTATU                 int64
HLTHPLAN                int64
QTY                     int64
RX                      int64
YEAR                    int64
PROC1                  string
SVCDATE                object
TSVCDAT                object
DX1                    string
DX2                    string
DX3                    string
DX4                    string
CLAIM                 float64
TIME                  float64
PROC1_CATEGORIES       string
DIAGNOSIS_OVERLAY       int64
dtype: object

In [22]:
# Take all the processed data into the relative csv files.
df.to_csv('10m_each_processed.csv')
df.to_csv('1m_each_processed.csv')
df.to_csv('data_df_15million.csv')